In [1]:
from azure.ai.ml import load_component
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import MLClient, Input
from azure.identity import DefaultAzureCredential, EnvironmentCredential
from azure.ai.ml.entities import AmlCompute
from dotenv import load_dotenv
import os
import pandas as pd

In [2]:
print("Cargar username/pass desde archivo .env")
load_dotenv("env.txt")

Cargar username/pass desde archivo .env


True

In [3]:
def get_comput_target(ml_client, name="cpu-cluster", family='Standard_DS2_v2'):
    cpu_compute_target = name
    
    try:
        # let's see if the compute target already exists
        cpu_cluster = ml_client.compute.get(cpu_compute_target)
    except Exception:
        cpu_cluster = AmlCompute(
            name=cpu_compute_target,
            type="amlcompute",
            size=family,
            min_instances=0,
            max_instances=4,
            idle_time_before_scale_down=180,
            tier="Dedicated",
        )
    
        cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster).result()

In [4]:
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential=credential)

Found the config file in: .\config.json


In [5]:
credential.get_token("https://management.azure.com/.default")

AccessToken(token='eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IjlHbW55RlBraGMzaE91UjIybXZTdmduTG83WSIsImtpZCI6IjlHbW55RlBraGMzaE91UjIybXZTdmduTG83WSJ9.eyJhdWQiOiJodHRwczovL21hbmFnZW1lbnQuYXp1cmUuY29tIiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvODRjYmMyMWItZmRmMy00MjcyLWFkMzQtZjRjMGZjZjcwZmExLyIsImlhdCI6MTY5OTE0MzUyNSwibmJmIjoxNjk5MTQzNTI1LCJleHAiOjE2OTkxNDc0MjUsImFpbyI6IkUyRmdZRmgyOThCaHJ3RDErdm81SDNhSmVvdE1BUUE9IiwiYXBwaWQiOiIwODg0MjVjNS01NDkxLTQyMWUtYmUyMy0wMmEyZjAwODc5MjciLCJhcHBpZGFjciI6IjEiLCJpZHAiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC84NGNiYzIxYi1mZGYzLTQyNzItYWQzNC1mNGMwZmNmNzBmYTEvIiwiaWR0eXAiOiJhcHAiLCJvaWQiOiJiODc2NDFkMy0yOTUwLTQwNzQtYTI2OS01YjI2ODhjNjczOTMiLCJyaCI6IjAuQWIwQUc4TExoUFA5Y2tLdE5QVEFfUGNQb1VaSWYza0F1dGRQdWtQYXdmajJNQlBMQUFBLiIsInN1YiI6ImI4NzY0MWQzLTI5NTAtNDA3NC1hMjY5LTViMjY4OGM2NzM5MyIsInRpZCI6Ijg0Y2JjMjFiLWZkZjMtNDI3Mi1hZDM0LWY0YzBmY2Y3MGZhMSIsInV0aSI6InNtVjVmcUhPMFV1d3FIV2FYZzVSQUEiLCJ2ZXIiOiIxLjAiLCJ4bXNfY2FlIjoiMSIsInhtc190Y2R0IjoxNjk1MjU5MzQxfQ.FXkaeJL2-VU1ODt_m

In [6]:
compute_target = get_comput_target(ml_client)

In [7]:
# TODO Add the rest of the components
train_component = load_component(source="./clean-data-component/clean-data.yml")

ValidationException: Target yaml file is empty